# Tutorial 1: Introduction to the `SQuADDS` Project

In this tutorial, we will walk you through some basic usage of SQuADDS. 

By the end of this tutorial, you will be able to:

- Access and understand the SQuADDS Database

- Use the SQuADDS API to query for closest and "best-guess" interpolated device designs for your chosen Hamiltonian parameters

In [1]:
%load_ext autoreload
%autoreload 2

## Installation and Setup

Please follow the [installation guide](https://lfl-lab.github.io/SQuADDS/source/getting_started.html) to install `SQuADDS` and set up the environment.

In [ ]:
import squadds

squadds.__version__

## The SQuADDS Database Structure on HuggingFace 🤗

HuggingFace datasets are structured as a collection of `configurations`. Each configuration in the dataset is uniquely identified by their `config`. For the SQuADDS Database, the `config` string is created in the following format:

```python
config = f"{component}_{component_name}_{data_type}"
```

where `component` is the name of the component, `component_name` is the name of the component (in Qiskit Metal), and `data_type` is the type of simulation data that has been contributed. 

This structured approach ensures that users can query specific parts of the dataset relevant to their work, such as a particular type of qubit design or simulation results. This API abstraction allows for more complex queries and operations on the data, facilitating a more efficient workflow for researchers and developers.

Lets check what the `config` string looks like for our database using the `SQuADDS` API.

## Using the SQuADDS API to access the database

While it is possible to directly access the SQuADDS Database using the `datasets` library, we have created a simple API to make it easier to query the database.

The main object we use to access the database is the `SQuADDS_DB` class. 

In [ ]:
from squadds import SQuADDS_DB

db = SQuADDS_DB()

To check for the `config` names

In [ ]:
db.get_configs()

You can get a summary of the datasets by running.

In [ ]:
db.view_datasets()

**NOTE: `'coupler-NCap-cap_matrix'` and `''coupler-CapNInterdigitalTee-cap_matrix'` are the same datasets. We will support them both since future releases will deprecate the term `NCap` and replace it with `CapNInterdigitalTee`.**

If you are interested to learn more about each configuration, you can do so by using the `get_dataset_info` method.


In [ ]:
db.get_dataset_info(component="qubit", component_name="TransmonCross", data_type="cap_matrix")

You can get the entire dataset as a Pandas DataFrame by using the `get_dataset` method.

In [ ]:
db.see_dataset(component="qubit", component_name="TransmonCross", data_type="cap_matrix")

If you want to learn about the who contributed the simulation data, you can use the following methods:

In [ ]:
db.view_sim_contributors_of("qubit", "TransmonCross", "cap_matrix")

You can also learn more about the measured device that generated this dataset:

In [ ]:
db.view_reference_device_of(component="qubit", component_name="TransmonCross", data_type="cap_matrix")

To learn more about the other measured devices that are available in SQuADDS

In [ ]:
db.get_measured_devices()

If you have data on a measured device that you want to contribute. We have a simple workflow API to do so. 


```python
from squadds.database.github import login_to_github, contribute_measured_data

# Authenticate with GitHub using your token from the .env file
github = login_to_github()

# Contributing measured data to the SQuADDS Database
new_device_entry = {}
pr_title="Adding DEVICENAME to SQuADDS_DB"

contribute_measured_data(new_device_entry, pr_title)
```

Please go over [Tutorial 4](https://lfl-lab.github.io/SQuADDS/source/tutorials/Tutorial_4_Contributing_Measured_Data_to_SQuADDS.html) on our docsite to learn more about this in details.

To see the list of all the contributors, you can use the following method:

In [ ]:
db.view_all_contributors()

As the `SQuADDS_DB` dataset updates, so will all the information we have queried automatically. 

## Making Systems out of Circuit QED Elements

One of the main use cases of the `SQuADDS` project is to get the design space parameters for systems of our interest using our desired Hamiltonian parameters.

Using the `SQuADDS` API, we can "build" a system by choosing the circuit QED components we want to use.

The following subsections walks you through some examples.

### Querying for the a target qubit design

Let's say you know the Hamiltonian parameters of a qubit you want to use. You can use the `SQuADDS` API to query for the closest design to your target qubit.

We first need to select our sytem of interest.

In [13]:
db.select_system("qubit")

Now, we need to specify to SQuADDS what type of `qubit` our system is. We can do this by using the `select_qubit` method.

In [16]:
db.select_qubit("TransmonCross")

We now create the system dataframe so that we can query for the design parameters we are interested in.

In [ ]:
df = db.create_system_df()
df

Now that we have created our system dataframe, we can query for the closest design to our target qubit parameters. To do this we need to call the `Analyzer` object.

In [18]:
from squadds import Analyzer

We instatantaite the `Analyzer` object by passing in the `SQuADDS_DB` instance we created earlier.

In [19]:
analyzer = Analyzer(db)

We can now check for what type of Hamiltonian parameters are available for our chosen system

In [ ]:
analyzer.target_param_keys()

Now, lets choose the Hamiltonian parameters that you want to use for your qubit 

In [21]:
target_params={"qubit_frequency_GHz": 4, "anharmonicity_MHz": -200}

To select a geometry which results in the closest qubit characteristics

Call `Analyzer.find_closest` which takes as arguments the target parameters, the number of closest designs to return, and a metric to use for the comparison.

You are given the choice of the following metrics.

In [ ]:
analyzer.__supported_metrics__

In [ ]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=3,
                                       metric="Euclidean",
                                       )
results

Thats it! You have now found some designs for your qubit that are closest to your target Hamiltonian parameters.

It is important to note that this returned dataset is **complete** in the sense that it contains information of both the Hamiltonian space parameters and its accompanying design space geometries.

#### Using Custom Metrics

To use a custom metric first define the function. For example, lets say we want the manhattan metric

In [24]:
def manhattan_distance(target, simulated):
    return sum(abs(target[key] - simulated.get(key, 0)) for key in target)

In [25]:
analyzer.custom_metric_func = manhattan_distance

In [ ]:
results = analyzer.find_closest(target_params=target_params,
                                            num_top=1,
                                            metric="Custom",
                                            )

In [ ]:
results

In [ ]:
best_options = results.iloc[0]["design_options"]
best_options

You can pass in the `design_options` from the closest design to the `options` argument of your selected qubit and render it in qiskit metal.

In [29]:
# Qiskit Metal imports
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict
from qiskit_metal.designs.design_multiplanar import MultiPlanar

from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.core import QRoute, QRoutePoint

In [ ]:
design = MultiPlanar(metadata={},
                     overwrite_enabled=True)
gui = MetalGUI(design)

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross

TransmonCross(design, "transmon", options=best_options)

gui.rebuild()
gui.zoom_on_components(['transmon'])
gui.screenshot("figures/qubit_only.png")

### Querying for a target qubit-cavity design


While it is not necessary, it may be a good idea to `unselect_all()` before creating a new system.

In [32]:
db.unselect_all()

Again, we follow the same procedure as before.

In [33]:
db.select_system(["qubit","cavity_claw"])

In [34]:
db.select_qubit("TransmonCross")
db.select_cavity_claw("RouteMeander")
db.select_resonator_type("quarter")

It's always a good idea to check that the system you have selected is correct.

In [ ]:
db.show_selections()

Great! lets create the system dataframe and analyze it.

In [ ]:
merged_df = db.create_system_df()

In [ ]:
merged_df

Pass the `SQuADDS_DB` instance to the `Analyzer` object.

In [34]:
analyzer = Analyzer(db)

Alternatively, you could do.

In [38]:
analyzer.reload_db()

Always good to check whether the system you have selected is correct.

In [ ]:
db.selected_system

In [ ]:
analyzer.selected_system

Define the `target_params` for your qubit-cavity system.

In [41]:
target_params = {
                "qubit_frequency_GHz": 4,
                "cavity_frequency_GHz": 8.9,
                "kappa_kHz": 220,
                "resonator_type":"quarter",
                "anharmonicity_MHz": -200,
                "g_MHz": 70}

In [ ]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=1,
                                       metric="Euclidean",
                                       )
results

Awesome! we have some designs for our qubit-cavity system. To see where the closest design lies in the Hamiltonian parameter space, we can use the `closest_design_in_H_space` method.

In [43]:
%matplotlib inline

In [ ]:
analyzer.closest_design_in_H_space()

## Interpolation of Best Designs

Even though the `closest_design` will become better as more validated pre-simulated points are added to the database, it is still a good idea to interpolate to get the best designs.

We use the physics inspired interpolation algorithm described in our [paper](https://arxiv.org/pdf/2312.13483.pdf) - `ScalingInterpolator` class to interpolate the best designs.

In [45]:
from squadds.interpolations.physics import ScalingInterpolator

We pass the `Analzyer` object and the `target_params` dict to the `ScalingInterpolator` class.

In [ ]:
# Create an instance of ScalingInterpolator
interpolator = ScalingInterpolator(analyzer, target_params)

design_df = interpolator.get_design()

The `design_df` contains the various `design_options` for the best designs and also the `sim_options` needed to simulate them.

In [ ]:
design_df

In [ ]:
design_df.iloc[0]["setup_cavity_claw"]

Let's use this interpolated deisgn to generate a `.gds` file.

In [48]:
design_options = design_df.iloc[0]["design_options"]

In [ ]:
design_options

In [ ]:
from squadds.components.coupled_systems import QubitCavity
import qiskit_metal as metal
from qiskit_metal import Dict, MetalGUI, designs, draw
from qiskit_metal.toolbox_metal import math_and_overrides

design = metal.designs.design_planar.DesignPlanar()
gui = metal.MetalGUI(design)
design.overwrite_enabled = True

qc_ncap = QubitCavity(design, "qubit_cavity", options=design_options)
gui.rebuild()
gui.autoscale()
gui.screenshot("figures/qubit_half_wave_cavity")

In [ ]:
qc_ncap.show(gui, include_wirebond_pads=True, figure_name="figures/qubit_cavity_t_line.png")

In [ ]:
qc_ncap.to_gds("gds_files/qubit_cavity", include_wirebond_pads=True)

Congrats for making it to the end of this tutorial! 🤗🎉 You have now learned how to use the SQuADDS API to query for closest and "best-guess" interpolated device designs for your chosen Hamiltonian parameters.

## Next Steps...

In the next tutorial, we will show you how to create a new chip design that is ready to be fabricated using the `SQuADDS` API.

<div style='width: 100%; background-color:#3cb1c2;color:#324344;padding-left: 10px; padding-bottom: 10px; padding-right: 10px; padding-top: 5px'>
    <h3>This SQuADDS tutorial was prepared for the Qiskit Fall Fest 2024</h3>
    <p>Developed by Sadman Ahmed Shanto</p>
    <p>This tutorial is written by Sadman Ahmed Shanto</p> 
    <p>&copy; Copyright Sadman Ahmed Shanto & Eli Levenson-Falk 2024.</p>
</div>